In [ ]:
import os
import os.path as osp
import sys
import numpy as np
import torch, torchvision
from torch import nn
from torch.utils import model_zoo
from torch.nn.parallel import DataParallel
from torch.utils.data import DataLoader
from torch.utils.data.dataset import ConcatDataset
# %load_ext autoreload
# %autoreload 2

In [ ]:
# Get cfg
import mmcv
config_file = 'visdrone/configs/ssd300.py'
cfg = mmcv.Config.fromfile(config_file)

In [ ]:
# Get model
from mmdet.models import build_detector
model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
# load from checkpoint
from mmcv.runner import load_checkpoint
checkpoint_file = 'work_dirs/ssd300_visdrone/latest.pth'
_ = load_checkpoint(model, checkpoint_file)

In [ ]:
from mmdet.apis import inference_detector, show_result

# test a single image and show the results
img = '/tmp/visdrone-debug/images/0000001_02999_d_0000005.jpg'
result = inference_detector(model, img, cfg, device='cuda')
# result is list of np array with [N, 5]
# show_result(img, result, 'visdrone')

# First visualize coco dataset

In [ ]:
import sys
sys.path.append('/home/damon/detorch/')
from visualization_utils import *
from debug_utils import *

In [ ]:
# Conver json['annotations'] to boxes
def get_gt_boxes_from_img_name_COCO(json_ann, image_name):
    image_id = None
    for im in json_ann['images']:
        if im['file_name'] != image_name:
            continue
        else:
            image_id = im['id']
            break
    print(image_id)
    if image_id == None:
        raise ValueError("Not found")
    boxes = []
    for info in json_ann['annotations']:
        if info['image_id'] == image_id:
            x1, y1, w, h = info['bbox']
            bbox = [y1, x1, y1 + h, x1 + w]
            boxes.append(bbox)
    return np.asarray(boxes)

In [ ]:
coco_imgs = '/home/damon/DATASETS/COCO/val2017/'
coco_anno_file = '/home/damon/DATASETS/COCO/annotations/instances_val2017.json'
coco_annos = mmcv.load(coco_anno_file)
images = coco_annos['images']
annotations = coco_annos['annotations']
all_ids = [q['image_id'] for q in annotations]
id2fname = {v['id']: v['file_name'] for v in coco_annos['images'] if v['id'] in all_ids}
fname2id = {fn: id for id, fn in id2fname.items()}
# find name by id
ind = np.random.randint(len(id2fname.keys()))
example_img = id2fname[all_ids[ind]]
print(example_img)
example_img_full_path = osp.join(coco_imgs, example_img)
boxes = get_gt_boxes_from_img_name_COCO(coco_annos, example_img)

In [ ]:
img = mmcv.imread(example_img_full_path)

In [ ]:
draw_bounding_boxes_on_image_array(img, boxes)
visualize_image_array(img)

In [ ]:
example_img

# Vissualize visdrone dataset

In [ ]:
import sys
sys.path.append('/home/damon/detorch/')
from visualization_utils import *
from debug_utils import *

In [ ]:
ann = '/tmp/visdrone-debug/annotations_debug.json'
gts = mmcv.load(ann)

In [ ]:
img_name = '0000001_02999_d_0000005.jpg'
img_full = osp.join('/tmp/visdrone-debug/images', img_name)
img_arr = mmcv.imread(img_full)
img_name = osp.join('images', img_name)
boxes = get_gt_boxes_from_img_name(gts, img_name)

In [ ]:
draw_bounding_boxes_on_image_array(img_arr, boxes)
visualize_image_array(img_arr)